In [1]:
import sqlite3

# Создание базы данных и таблиц
conn = sqlite3.connect('example.db')
c = conn.cursor()

c.execute('''CREATE TABLE City
             (Id INTEGER PRIMARY KEY, Name TEXT)''')

c.execute('''CREATE TABLE Person 
             (Name TEXT, CityId INTEGER, FOREIGN KEY(CityId) REFERENCES City(Id))''')

# Добавление данных в таблицы
c.execute("INSERT INTO City VALUES (1, 'Москва')")
c.execute("INSERT INTO City VALUES (2, 'Санкт-Петербург')")
c.execute("INSERT INTO City VALUES (3, 'Казань')")

c.execute("INSERT INTO Person VALUES ('Андрей', 1)")
c.execute("INSERT INTO Person VALUES ('Леонид', 2)")
c.execute("INSERT INTO Person VALUES ('Сергей', 1)")
c.execute("INSERT INTO Person VALUES ('Григорий', 4)")

conn.commit()

# Примеры join-запросов
print("INNER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             INNER JOIN City
             ON Person.CityId = City.Id''')
print(c.fetchall())

print("\nLEFT OUTER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             LEFT OUTER JOIN City
             ON Person.CityId = City.Id''')
print(c.fetchall())

print("\nRIGHT OUTER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             RIGHT OUTER JOIN City
             ON Person.CityId = City.Id''')
print(c.fetchall())

print("\nFULL OUTER JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             FULL OUTER JOIN City
             ON Person.CityId = City.Id''')
print(c.fetchall())

print("\nCROSS JOIN:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             CROSS JOIN City''')
print(c.fetchall())

print("\nLEFT JOIN with NULL check:")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             LEFT JOIN City
             ON Person.CityId = City.Id
             WHERE City.Id IS NULL''')
print(c.fetchall())

print("\nRIGHT JOIN with NULL check (эмулируется с помощью LEFT JOIN):")
c.execute('''SELECT Person.Name, City.Name
             FROM Person
             LEFT JOIN City
             ON Person.CityId = City.Id
             WHERE Person.CityId IS NULL''')
print(c.fetchall())

conn.close()


INNER JOIN:
[('Андрей', 'Москва'), ('Леонид', 'Санкт-Петербург'), ('Сергей', 'Москва')]

LEFT OUTER JOIN:
[('Андрей', 'Москва'), ('Леонид', 'Санкт-Петербург'), ('Сергей', 'Москва'), ('Григорий', None)]

RIGHT OUTER JOIN:
[('Андрей', 'Москва'), ('Леонид', 'Санкт-Петербург'), ('Сергей', 'Москва'), (None, 'Казань')]

FULL OUTER JOIN:
[('Андрей', 'Москва'), ('Леонид', 'Санкт-Петербург'), ('Сергей', 'Москва'), ('Григорий', None), (None, 'Казань')]

CROSS JOIN:
[('Андрей', 'Москва'), ('Андрей', 'Санкт-Петербург'), ('Андрей', 'Казань'), ('Леонид', 'Москва'), ('Леонид', 'Санкт-Петербург'), ('Леонид', 'Казань'), ('Сергей', 'Москва'), ('Сергей', 'Санкт-Петербург'), ('Сергей', 'Казань'), ('Григорий', 'Москва'), ('Григорий', 'Санкт-Петербург'), ('Григорий', 'Казань')]

LEFT JOIN with NULL check:
[('Григорий', None)]

RIGHT JOIN with NULL check (эмулируется с помощью LEFT JOIN):
[]
